In [ ]:
import nltk
nltk.download('stopwords')
!pip install pytypo
!pip install spellchecker
!pip install pyspellchecker
!pip install bs4
!pip install tqdm
!pip install spacy-symspell

In [ ]:
import pandas as pd  
import numpy as np
cols = ['notknow','text','target','id']
cleaned_tweets = pd.read_csv("/kaggle/input/clean-sentiment140/clean_tweets.csv",header=None, names=cols, encoding = 'ISO-8859-1', low_memory=False)
# above line will be different depending on where you saved your data, and your file name
cleaned_tweets.drop(columns=['notknow'], inplace=True)
cleaned_tweets.drop(cleaned_tweets.index[0], inplace=True)
cleaned_tweets.reset_index(inplace=True)
cleaned_tweets.drop(columns=['index'], inplace=True)
cleaned_tweets.head()

In [ ]:
from spellchecker import SpellChecker
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
import pytypo
spell = SpellChecker()
spell.word_frequency.load_words(['microsoft', 'apple', 'google','twitter', 'facebook', 'amazon', 'texting', 'nah'])
stop_words = set(stopwords.words('english')) 
contractions = ['ive', 'ya', 'b', 'ill', 'yall', 'im', 'youre', 'youll', 'theyre', 'didnt', 'cant', 'dont', 'howd', 'id', 'isnt', 'shouldnt', 'wheres', 'wont', 'wouldve', 'couldnt', 'hadnt', ' hasnt']
for stop_w in contractions:
    stop_words.add(stop_w)


def correctSpelling(word_tokens):
    
    if(len(word_tokens) > 0):
        misspelled = spell.unknown(word_tokens)
        if(len(misspelled) > 0):
            words_dict = dict()
            for i in range(len(word_tokens)):
                if word_tokens[i] in words_dict:
                    words_dict[word_tokens[i].lower()].append(i)
                else:
                    words_dict[word_tokens[i].lower()] = [i]

            for word in misspelled:
                word_corrected = spell.correction(word)
                for index in words_dict[word]:
                    if(word_corrected == word):
                        word_corrected = pytypo.correct(word)
                    word_tokens[index] = word_corrected
            # STOP WORDS
        filtered_sentence = [w for w in word_tokens if not w in stop_words]

    return (" ".join(filtered_sentence)).strip()

In [ ]:
%%time

from tqdm import tqdm
fromtweet = 610000
tok = WordPunctTokenizer()
correct_tweet_texts = []
correct_tweet_targets = []
correct_tweet_ids = []
print ("Correcting spelling for tweets...\n")
for i in tqdm(range(fromtweet,fromtweet+10000)):
    if type(cleaned_tweets['text'][i]) is str:
        clean_tuit = tok.tokenize(cleaned_tweets['text'][i])
        #print(correctSpelling(clean_tuit))
        correct_tweet_texts.append(correctSpelling(clean_tuit))
        correct_tweet_targets.append(cleaned_tweets['target'][i])
        correct_tweet_ids.append(cleaned_tweets['id'][i])

cleaner_df = pd.DataFrame(correct_tweet_texts,columns=['text'])
cleaner_df['target'] = correct_tweet_targets
cleaner_df['id'] = correct_tweet_ids
cleaner_df.to_csv('clean_corrected_tweets_'+ str(fromtweet) +'.csv',encoding='utf-8',index=False)